In [ ]:
!pip install flask_jwt_extended
!pip install flask


  Using cached MarkupSafe-2.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached MarkupSafe-2.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# Read csv files

In [ ]:
import pandas as pd

lab_df =  pd.read_csv("lab_results_10_tests_patients_1_15_NA.csv")
lab_df['date'] = pd.to_datetime(lab_df['date'])


patient_df =  pd.read_csv("patient_information.csv")



## Functions

### Check login / get patient id from user id

In [ ]:
def check_login(input_user, input_pass, df):
    if input_user in patient_df.username.values:

        password = df[df.username==input_user]['password'].item()
        
        if input_pass == password:
            print('Login successful')
            return True
        else:
            print('Wrong password')
            return False

    else:
        print('Username not available')
        return False



    


### From username to patient_id

In [ ]:
def patient_id (user_id, df):
    id_pat = df[df['username']==user_id]['patient_id'].item()
    return id_pat

### Get lab data for specific patient_id

In [ ]:
def get_lab_data(pat_id, df):
    patient11 = df[df.patient_id == pat_id]
    patient11 = patient11.copy()
    patient11.sort_values(by='date', inplace=True)
    patient11.reset_index(inplace=True, drop=True)

    return patient11

# Flask API - EHR

In [ ]:
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, jwt_required, create_access_token, get_jwt_identity
from json import loads, dumps

app = Flask(__name__)
app.config['JWT_SECRET_KEY'] = 'tmObbSxOOwIwh12IbYIM8Ouojt14FrZEnQ4JEJbluVxastT6HD3YKMldFvRUv6P'
jwt = JWTManager(app)
 
# The route() function of the Flask class is a decorator, 
# which tells the application which URL should call 
# the associated function.





@app.route('/login', methods=['POST'])
def login():
    if request.is_json:
        user = request.json['user']
        password = request.json['password']
    else:
        user = request.form['user']
        password = request.form['password']

    
    if check_login(user, password, patient_df):
        access_token = create_access_token(identity=user)
        return jsonify(message='Login Successful', access_token=access_token)
    else:
        return jsonify('Bad email or Password'), 401







@app.route('/', methods=['GET'])
@jwt_required()
def hello_world():
    user_id = get_jwt_identity()
    print(user_id)

    return "Welcome to EHR database, you're logged in as: " + user_id





#lab data

@app.route('/lab_data', methods=['GET'])
@jwt_required()
def lab_data():
    
    user_id = get_jwt_identity()
    patient_nr = patient_id(user_id, patient_df)
    data = get_lab_data(patient_nr, lab_df)
    data_json = data.to_json(orient='records')

    parsed = loads(data_json)
    obj = {
        'patients': parsed
        }

    return obj, 200






#patient basic information

@app.route('/patient_data', methods=['GET'])
@jwt_required()
def patient():
    user_id = get_jwt_identity()
    data = patient_df[patient_df.username == user_id].to_json(orient='records')
    parsed = loads(data)
    obj = {
        'patients': parsed
        }

    return obj, 200





app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.62.215:8080
Press CTRL+C to quit
172.3.27.152 - - [30/Nov/2023 15:49:49] "POST /login HTTP/1.1" 200 -
172.3.27.152 - - [30/Nov/2023 15:49:49] "GET /patient_data HTTP/1.1" 200 -
172.3.27.152 - - [30/Nov/2023 15:49:49] "GET /lab_data HTTP/1.1" 200 -
Login successful


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c719ecc6-7add-42e5-96f8-239e9c0d87b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>